In [ ]:
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
session = get_active_session()
df = session.table('ZENTIME_CUSTOMER_SUPPORT_TICKETS')
df

In [ ]:
prompt = """You are an agent that determines the category of an issue based on the ISSUE_DESCRIPTION 
for smartwatches.

These are the ONLY possible categories:
- Technical Issues
- Billing Issues
- Delivery Related / Missing Items
- Battery Problem
- Product Issues

Given an ISSUE_DESCRIPTION, return only one category from the list above. 
Here is a description for each category that you could use to categorize.

Billing Issues: Subscription payment failures, refund requests, discount code problems, 
warranty coverage questions, and unexpected charges

Technical Issues: App connectivity failures, software update errors, synchronization problems, 
Bluetooth pairing difficulties, and account access problems

Delivery Related / Missing Items: Shipping delays, lost packages, incomplete orders, incorrect delivery addresses,missing accessories, 
incorrect items such as wrong colour watch or specifications than what was requested

Battery Problem: Short battery life, failure to hold charge, overheating while charging, unexpected shutdowns,
fluctuating battery performance, and charging cable malfunctions

Product Issues: Physical damage, screen malfunctions, unresponsive buttons, water damage claims.

Strictly do not return anything else."""


## Use mistral-large2

In [ ]:
mistral_large2_response_sql = f""" select support_ticket_id, ISSUE_DESCRIPTION,
trim(snowflake.cortex.complete('mistral-large2',concat('{prompt}',ISSUE_DESCRIPTION)),'\n') 
as mistral_large2_response
from zentime_customer_support_tickets"""
df_mistral_large2_response = session.sql(mistral_large2_response_sql)
df_mistral_large2_response

## Use mistral-7b

In [ ]:
mistral_7b_response_sql = f""" select support_ticket_id, ISSUE_DESCRIPTION,
trim(snowflake.cortex.complete('mistral-7b',concat('{prompt}',ISSUE_DESCRIPTION)),'\n') as mistral_7b_response
from zentime_customer_support_tickets"""
df_mistral_7b_response = session.sql(mistral_7b_response_sql)
df_mistral_7b_response

## Compare LLMs - mistral_large2 and mistral-7b

In [ ]:
df_llms = df_mistral_large2_response.join(df_mistral_7b_response,'SUPPORT_TICKET_ID')
df_llms[['SUPPORT_TICKET_ID', 'mistral_large2_response','mistral_7b_response']]

# Generate dataset to fine-tune mistral-7b

In [ ]:
df_fine_tune = df_mistral_large2_response.with_column(
    "prompt",
    F.concat(
        F.lit(prompt), F.lit(" "),
        F.col("ISSUE_DESCRIPTION"), F.lit(" ")
    )
).select("SUPPORT_TICKET_ID", "prompt", "mistral_large2_response")
# df_fine_tune
df_fine_tune.write.mode('overwrite').save_as_table('zentime_customer_support_tickets_finetune')


# Fine-tune mistral-7b

## Split data into training and evaluation

In [ ]:
train_df, eval_df = session.table("zentime_customer_support_tickets_finetune").random_split(weights=[0.8, 0.2], 
                                                                                            seed=42)
train_df.write.mode('overwrite').save_as_table('zentime_customer_support_train')
eval_df.write.mode('overwrite').save_as_table('zentime_customer_support_eval')

## *Fine-tune mistral-7b model using Cortex*

In [ ]:
select SNOWFLAKE.CORTEX.FINETUNE('CREATE', 'CORTEX_AI_DB.PUBLIC.zentime_customer_support_FINETUNED_mistral_7b_model','mistral-7b', 'SELECT prompt, mistral_large2_response as completion from CORTEX_AI_DB.PUBLIC.zentime_customer_support_train', 'SELECT prompt, mistral_large2_response as completion from CORTEX_AI_DB.PUBLIC.zentime_customer_support_eval')zentime_fine_tuned_job_id;

### Copy the job id from above and feed it in the below Query to see status of the fine tuning job

In [ ]:
select snowflake.cortex.finetune('DESCRIBE', 'ft_f8c4e8d4-c65b-4b8c-9f80-75a96fbcef42');

# Inference using fine-tuned model

In [ ]:
fine_tuned_model_name = 'zentime_customer_support_FINETUNED_mistral_7b_model'

sql = f"""select support_ticket_id, issue_description,
trim(snowflake.cortex.complete('{fine_tuned_model_name}',concat('{prompt}',
issue_description)),'\n') as zt_fine_tuned_mistral_7b_model_response
from zentime_customer_support_tickets"""

df_fine_tuned_mistral_7b_response = session.sql(sql)
df_fine_tuned_mistral_7b_response